## Import Libraries 

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib qt
from sklearn import linear_model

## Load & Clean Data 

In [5]:
Data = pd.read_csv(r'NG_cleaned.csv')
current_weather = pd.read_csv(r'current_weather.csv', index_col=0, parse_dates=True)

Data.head()
Data = Data.dropna()
Data.isna().sum()

Date       0
Spot       0
Storage    0
HDD        0
CDD        0
dtype: int64

In [6]:
Data['TDDs'] = Data['HDD'] + Data['CDD']

In [7]:
current_weather.tail()

,HDD,CDD
2021-01-04,22.72,0.32
2021-01-05,28.07,0.49
2021-01-06,28.11,0.43
2021-01-07,24.36,0.42
2021-01-08,24.42,0.51


In [8]:
two_week_forecasted_weather = current_weather['HDD'][-1] + current_weather['CDD'][-1] 
two_week_forecasted_weather

24.930000000000003

In [9]:
newData = Data.append({'TDDs': two_week_forecasted_weather,
                       'Storage': 2978}, ignore_index=True)

newData.fillna(0)

newData.tail()

,Date,Spot,Storage,HDD,CDD,TDDs
3637,2020-12-22,2.76,3508.857143,21.0,0.0,21.00
3638,2020-12-23,2.76,3492.571429,21.0,0.0,21.00
3639,2020-12-24,2.68,3476.285714,25.0,0.0,25.00
3640,2020-12-25,2.62,3460.000000,28.0,0.0,28.00
3641,NaN,NaN,2978.000000,NaN,NaN,24.93


## Examining the Data 

In [23]:
fig, ax = plt.subplots()
ax = fig.add_subplot(111, projection='3d')
fig.set_size_inches(13,13)

x=Data['Storage']
y=Data['TDDs']
z=Data['Spot']

x = np.array(x)
y = np.array(y)
z = np.array(z)

ax.scatter(x, y, z, c=Data.index, marker='o', linewidths=0.5, edgecolors='blue')

plt.title('Price/TDDs/Storage')

ax.set_xlabel('Total Underground Storage')
ax.set_ylabel('TDDs per day')
ax.set_zlabel('Price')
ax.set_zlim(1,8)

plt.show()

## Fitting an OLS model 

In [12]:
Q = newData[['TDDs','Storage']][:-1]
R = newData['Spot'][:-1]

regr_model = linear_model.LinearRegression()
regr_model.fit(Q, R)

print('Intercept: \n', regr_model.intercept_)
print('Coefficients: \n', regr_model.coef_)

# prediction
New_TDDs = newData['TDDs'].iloc[-1]
New_Storage = newData['Storage'].iloc[-1]

print ('Predicted Price of Natural Gas: \n', regr_model.predict([[New_TDDs ,New_Storage]]))

Intercept: 
 3.5058125288737623
Coefficients: 
 [ 0.01819347 -0.00025702]
Predicted Price of Natural Gas: 
 [3.19398062]


## Adding a Regression Plane 

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from mpl_toolkits.mplot3d import Axes3D

# Data reshaping

X = Data[['TDDs', 'Storage']].values.reshape(-1,2)
Y = Data['Spot']

# Prepare model data point for visualization

x = X[:, 0]
y = X[:, 1]
z = Y

New_TDDs = newData['TDDs'].iloc[-1]
New_Storage = newData['Storage'].iloc[-1]

x_pred = np.linspace(0, 45, 120)   # range of porosity values
y_pred = np.linspace(800, 4050, 120)  # range of brittleness values
xx_pred, yy_pred = np.meshgrid(x_pred, y_pred)
model_viz = np.array([xx_pred.flatten(), yy_pred.flatten()]).T

# Train model

ols = linear_model.LinearRegression()
model = ols.fit(X, Y)
predicted = model.predict(model_viz)

# Evaluate model

r2 = model.score(X, Y)

# Plot 

plt.style.use('default')

fig, ax = plt.subplots()
ax = fig.add_subplot(111, projection='3d')
fig.set_size_inches(13,13)

ax.scatter(New_TDDs, New_Storage, 2.603, c='red', marker='o', linewidths=1, edgecolors='black', s=50)
ax.scatter(x, y, z, c='springgreen', marker='o', linewidths=0.5, edgecolors='blue', s=5)
ax.scatter(xx_pred.flatten(), yy_pred.flatten(), predicted, facecolor=(0,0,0,0), s=0.45, c='red', linewidths=0.08, edgecolor='red')
ax.set_xlabel('TDDs', fontsize=12)
ax.set_ylabel('Storage', fontsize=12)
ax.set_zlabel('Price', fontsize=12)
ax.set_xlim(0,50)

fig.suptitle('Price/TDDs/Storage with Regression Plane', fontsize=20)
#fig.suptitle('$R^2 = %.2f$' % r2, fontsize=20)

fig.tight_layout()